In [29]:
import pandas as pd
import numpy as np
import re
import requests
import searchtweets
from textblob import TextBlob, Blobber
from textblob_fr import PatternTagger, PatternAnalyzer
from datetime import *

In [45]:
df_tweets = pd.read_csv("~/work/projetpy/fichiers_csv/tweets_2021-3-1_to_2021-4-1.csv")
df_tweets.head()

,id,date,screenName,text,type
0,1377376779178967041,2021-03-31 21:46:29,ratonlave57,Pourquoi ne donne t’on pas de l’Aspegic pédiat...,tweet
1,1377339517170933768,2021-03-31 19:18:25,MathildeSlv,@mvrinesv Le syndrome grippal du Pfizer existe...,tweet
2,1377337776819990528,2021-03-31 19:11:30,EBrussat,@montagnethiers #Orleat 210 personnes #vaccine...,tweet
3,1377336854379331584,2021-03-31 19:07:50,garrec_pierre,"En tout cas vendredi, c’est le vaccin astrazen...",tweet
4,1377325612134699012,2021-03-31 18:23:10,FPasquisDumont,1700 centres Pfizer et Moderna...\nPlus de 70 ...,tweet


# Nettoyage de texte

# Construction des label avec TextBlob

Il existe deux méthodes pour construire des sentiments avec TextBlob

In [47]:
text = "quel beau temps"

In [48]:
blob = TextBlob(text, pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())
blob.sentiment

(0.8, 0.8)

In [49]:
tb = Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())
blob1 = tb(text)
blob1.sentiment

(0.8, 0.8)

In [28]:
df_tweets['sentiment_label'] = [tb(text).sentiment[0] for text in df_tweets.text]
df_tweets[['sentiment_label', 'text']].head()

,sentiment_label,text
0,0.000,Pourquoi ne donne t’on pas de l’Aspegic pédiat...
1,0.000,@mvrinesv Le syndrome grippal du Pfizer existe...
2,0.375,@montagnethiers #Orleat 210 personnes #vaccine...
3,0.220,"En tout cas vendredi, c’est le vaccin astrazen..."
4,0.138,1700 centres Pfizer et Moderna...\nPlus de 70 ...


# Création des métadonnées

In [ ]:
def decompte_hashtags(text):
    exp_match = re.compile("([#]\w+)")
    res = exp_match.findall(text)
    return len(res)

df_tweets["nbre_hashtags"] = df_tweets["text"].apply(lambda x : decompte_hashtags(x))

In [ ]:
def decompte_tags(text):
    exp_match = re.compile("([@]\w+)")
    res = exp_match.findall(text)
    return len(res)

df_tweets["nbre_tags"] = df_tweets["text"].apply(lambda x : decompte_tags(x))

In [ ]:
def decompte_mails(text):
    exp_match = re.compile("(\w+[@]\w+[.]\w+)")
    res = exp_match.findall(text)
    return len(res)

df_tweets["nbre_mails"] = df_tweets["text"].apply(lambda x : decompte_mails(x))

In [ ]:
def decompte_exclamation(text):
    exp_match = re.compile("(\w?\s?[!])")
    res = exp_match.findall(text)
    return len(res)

df_tweets["nbre_exclamation"] = df["text"].apply(lambda x : decompte_exclamation(x))

In [ ]:
def decompte_interrogation(text):
    exp_match = re.compile("(\w?\s?[?])")
    res = exp_match.findall(text)
    return len(res)

df_tweets["nbre_interrogation"] = df_tweets["text"].apply(lambda x : decompte_interrogation(x))

In [ ]:
def decompte_mot_maj(text):
    exp_match = re.compile("([A-Z][A-Z]+)")
    res = exp_match.findall(text)
    return len(res)

df_tweets["nbre_maj"] = df_tweets["text"].apply(lambda x : decompte_mot_maj(x))

In [ ]:
df_tweets.head()